# Planning

<style>
table {
  margin-left: auto;
  margin-right: auto;
}

td {
  text-align: center;
}
</style>

<table>
  <tr>
    <th></th>
    <th>day 1</th>
    <th>day 2</th>
    <th>day 3</th>
    <th>day 4</th>
    <th>day 5</th>
    <th>day 6</th>
    <th>day 7</th>
    <th>day 8</th>
    <th>day 9</th>
    <th>day 10</th>
  </tr>
  <tr>
    <th></th>
    <th>terça</th>
    <th>quarta</th>
    <th>quinta</th>
    <th>sexta</th>
    <th>sábado</th>
    <th>domingo</th>
    <th>segunda</th>
    <th>terça</th>
    <th>quarta</th>
    <th>quinta</th>
  </tr>
  <tr>
    <td>Literature review</td>
    <td style="text-align: center;">x</td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>EDA and pre processing</td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>Experiments</td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>Train</td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>Evaluate</td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>Inference</td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>Interpret</td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
  </tr>
</table>


# Setting environment

In [1]:
# verificar se o virtual env tá funcionando, mas acho melhor usar conda env
!virtualenv venv
!source venv/bin/activate
!pip3 install -q -r requirements.txt


created virtual environment CPython3.9.6.final.0-64 in 2316ms
  creator CPython3macOsFramework(dest=/Users/sayuriiwai/Library/CloudStorage/GoogleDrive-sayuri.iwai@gmail.com/Meu Drive/0 Work/Tech/repos/ecommerce-fraud-detection/venv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/Users/sayuriiwai/Library/Application Support/virtualenv)
    added seed packages: pip==23.1.2, setuptools==67.7.2, wheel==0.40.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [11]:
# Import libraries
# custom modules
from configs.custom_config import get_target_column, get_feature_columns, get_categorical_columns
from pipeline.preprocess import get_dataset, split_data, handle_missing_values, transform_categorical, scale_numerical
# external modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
pd.options.display.float_format = '{:.2f}'.format

In [12]:
main_df = get_dataset()

# EDA

In [13]:
describe_df = pd.DataFrame(columns=['attribute','null','type','unique_values','count'])

for column in main_df.columns:
    unique_values = main_df[column].unique()
    new_row = pd.Series({'attribute': column, 
                         'null': f'{round(main_df[column].isna().sum()/len(main_df) * 100, 2)} %', 
                         'type': main_df[column].dtype,
                         'unique_values': unique_values,
                         'count': len(unique_values)})
    describe_df.loc[len(describe_df)] = new_row

print(len(main_df))
describe_df

,attribute,null,type,count,unique_values
0,a,0.0 %,int64,4,"[4, 2, 1, 3]"
1,b,0.0866 %,float64,7673,"[0.7685, 0.755, 0.7455, 0.7631, 0.7315, 0.7359..."
2,c,0.0866 %,float64,135091,"[94436.24, 9258.5, 242549.09, 18923.9, 5728.68..."
3,d,0.0024 %,float64,52,"[20.0, 1.0, 3.0, 50.0, 15.0, 2.0, 6.0, 39.0, 1..."
4,e,0.0 %,float64,43208,"[0.4448275862068965, 0.0, 0.4823848238482384, ..."
5,f,0.0001 %,float64,1339,"[1.0, 33.0, 19.0, 18.0, 47.0, 0.0, 15.0, 41.0,..."
6,g,0.0013 %,object,52,"[BR, AR, UY, US, GB, nan, SE, FR, MX, RU, CL, ..."
7,h,0.0 %,int64,59,"[5, 0, 23, 2, 1, 11, 3, 24, 8, 17, 9, 7, 15, 2..."
8,i,0.0 %,object,127804,[Máquininha Corta Barba Cabelo Peito Perna Pel...
9,j,0.0 %,object,8324,"[cat_8d714cd, cat_64b574b, cat_e9110c5, cat_d0..."


In [ ]:
# Check for duplicates
columns_to_check = get_key_columns()
duplicates = main_df.duplicated(subset=columns_to_check, keep=False)
duplicate_rows = main_df[duplicates]
print(duplicate_rows)

#### Binary variables

In [ ]:
binary_columns = get_binary_columns()
binary_df = main_df[binary_columns]
binary_df

In [ ]:
for column in binary_columns:
    print(column)
    unique = main_df[column].unique()
    df_binary = pd.DataFrame(columns=['label','n','percentage'])
    for label in unique:
        n = main_df[column].value_counts()[label]
        new_row = pd.Series({'label': label,
                            'n': n,
                            'percentage': f'{round(n/len(main_df)*100,4)} %'})
        df_binary.loc[len(main_df)] = new_row
    print(df_binary)

#### Categorical variables

In [ ]:
categorical_cols = get_categorical_columns()
categorical_df = main_df[binary_columns]
categorical_df

In [ ]:
from scipy.stats import ttest_ind
cat_var = ''
target_column = get_target_column()
cat1 = main_df[main_df[cat_var] == 'X'][target_column]
cat2 = main_df[main_df[cat_var] == 'Y'][target_column]
t_statistic, p_value = ttest_ind(cat1, cat2)
if p_value < 0.05:
    print(f"Reject null hypothesis: {target_column} is different for the two categories")
else:
    print(f"Accept null hypothesis: {target_column} is the same for the two categories")

In [ ]:
target_df = main_df[(main_df[cat_var] == 'X') & (main_df[cat_var2] == 'Y')]
target_df.drop([cat_var, cat_var2], axis=1, inplace=True)

#### Numerical variables

In [ ]:
value_columns = get_value_columns()
value_df = main_df[value_columns]
value_df

In [ ]:
variable = ''
frequency_df = pd.DataFrame(main_df[variable].value_counts())
frequency_df.columns = ['Count']
frequency_df = frequency_df.sort_values('Count', ascending=False)
print(frequency_df)

In [ ]:
group_var = ''
variable = ''
main_df.groupby(group_var).value_counts([variable]).sort_values(ascending=False)

In [14]:
variable = ''
variable_unique = main_df[variable].unique()
df_unique = pd.DataFrame(columns=['label','n','percentage'])
for label in variable_unique:
    n = main_df[variable].value_counts()[label]
    new_row = pd.Series({'label': label,
                         'n': n,
                         'percentage': f'{round(n/len(main_df)*100,4)} %'})
    df_unique.loc[len(df_unique)] = new_row
df_unique

,label,n,percentage
0,0,142500,95.0 %
1,1,7500,5.0 %


### Explore missing values

In [ ]:
columns_with_missing_values = get_columns_with_missing_values()

for column in columns_with_missing_values:
    print(f'The min value for {column} is {main_df[column].min()}')
    print(f'The max value for {column} is {main_df[column].max()}')

In [15]:
variable = ''
grouped_df = main_df.groupby(variable)
null_counts = grouped_df.apply(lambda x: x.isnull().sum())
print(null_counts)

        a      b      c    d  e   f    g  h  i  j  k   l    m  n       o  p   
fraude                                                                        
0       0  12155  12155  336  0  11  179  0  0  0  0  11  336  0  106620  0  \
1       0    829    829   29  0   0   15  0  0  0  0   0   29  0    2237  0   

        fecha  monto  score  fraude  
fraude                               
0           0      0      0       0  
1           0      0      0       0  


In [ ]:
variable = ''
grouped_df = main_df.groupby(variable)
missing_values_count = grouped_df[columns_with_missing_values].apply(lambda x: x.isnull().sum())
print(missing_values_count.T)

In [ ]:
for variable in columns_with_missing_values:
    counts_df = main_df[variable].value_counts().sort_index()
    total_records = len(main_df)
    percentage_counts = (counts_df / total_records) * 100
    print(percentage_counts)

In [16]:
main_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
a,150000.0,3.705407,0.753206,1.000000,4.000000,4.000000,4.000000,4.000000e+00
b,137016.0,0.728115,0.132943,0.000000,0.678400,0.755500,0.806500,1.000000e+00
c,137016.0,260445.107044,846436.141626,0.160000,9679.915000,43711.655000,145443.627500,1.387874e+07
d,149635.0,21.677669,20.062146,0.000000,2.000000,14.000000,50.000000,5.000000e+01
e,150000.0,0.220641,2.434995,0.000000,0.000000,0.104875,0.282938,8.333333e+02
f,149989.0,51.169352,709.472904,-5.000000,1.000000,8.000000,33.000000,1.452740e+05
h,150000.0,14.193513,14.161216,0.000000,3.000000,9.000000,21.000000,5.800000e+01
k,150000.0,0.497532,0.288348,0.000004,0.246819,0.495990,0.746508,9.999948e-01
l,149989.0,2305.409403,1712.379601,0.000000,910.000000,1937.000000,3445.000000,7.544000e+03
m,149635.0,299.969579,321.075806,0.000000,42.000000,193.000000,459.000000,2.225000e+03


### Questions to be answered

### Data visualization

In [ ]:
variable = ''
group_var = ''
key_column = ''
var_counts = main_df.groupby(group_var)[variable].nunique().reset_index(name='COUNT').sort_values('COUNT', ascending=False)
occurrence_count = cycle_counts.groupby('COUNT')[variable].count().reset_index(name='OCCURRENCE_COUNT').sort_values('COUNT', ascending=False)
plt.figure(figsize=(15, 6))
sns.barplot(x='COUNT', y='OCCURRENCE_COUNT', data=occurrence_count)
plt.xlabel(f'{variable} COUNT')
plt.ylabel('OCCURRENCE_COUNT')
plt.title('Bar Chart')
plt.show()
main_df = main_df.merge(var_counts, on=key_column, how='left')

In [ ]:
import plotly.express as px

group_vars = []
for feature in value_columns:
    grouped_data = main_df.groupby(group_vars)[feature].mean()
    grouped_data = grouped_data.reset_index()

    fig = px.line(grouped_data, x=get_temporal_column(), y=feature, color=group_vars,
                  title=f"{feature} Over Time Grouped by {group_vars}")

    fig.show()

### Time series analysis

In [ ]:
from statsmodels.tsa.stattools import adfuller

ts_df = main_df
# ts_df = main_df[[get_temporal_column(), get_target_column()]]
# ts_df = ts_df.set_index(get_temporal_column())

result = adfuller(ts_df[get_target_column()])

p_value = result[1]

if p_value < 0.05:
    print("The time series is stationary.")
else:
    print("The time series is not stationary.")

In [ ]:
target_column = get_target_column()
temporal_column = get_temporal_column()
ts_df[temporal_column] = [datetime.datetime.strptime(str(date), '%Y-%m-%d %H:%M:%S') for date in main_df[temporal_column]]
ts_df = ts_df.sort_values(temporal_column)

mean_df = ts_df.groupby(temporal_column)[target_column].mean().reset_index()
mean_df = ts_df.sort_values(temporal_column)

mean_df[temporal_column] = pd.to_datetime(mean_df[temporal_column])
mean_df = mean_df.set_index(temporal_column)

y = mean_df[target_column].resample('M').mean()

import statsmodels.api as sm

decomposition = sm.tsa.seasonal_decompose(y, model='additive')
fig = decomposition.plot()
plt.show()

### Feature engineering

Define functions at preprocess script

In [ ]:
%%time
from tqdm import tqdm

unique = main_df[variable].unique()

    for id in tqdm(unique, desc="Processing", unit="id"):

### Correlation matrix

In [ ]:
from pipeline.preprocess import get_correlation_matrix

_drop_cols = get_correlation_matrix(main_df)

In [ ]:
drop_cols.remove([])
drop_cols.extend([])
main_df.drop(columns=drop_cols, inplace=True)

## Model Experiments